In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re

# Data import

In [ ]:
df_train=pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
sample=pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")

# Basic Checks

In [ ]:
print("No.of Row's and Columns of data \n *********************** \n  ",df_train.shape,
      "\n\n Top 5 records of the data \n ************************",df_train.head(5),
      "\n\n ************************** \n \n Bottom 5 records of the data \n\n ******************",df_train.tail(5))

In [ ]:
#creating id values as list for json data import
id_values=df_train.Id.values.tolist()

#Creating a function to import json data
idwise_json_txt=[]
def json_txt_extraction():    
    for id_lst in range(len(id_values)):
#         print(id_lst,id_values[id_lst])
        f = open("../input/coleridgeinitiative-show-us-the-data/train/"+ id_values[id_lst] +".json",)
        data = json.load(f)
        dict_txt_lst=[]
        for inp_txt in range(len(data)):
            txt=data[inp_txt]
            txtn=txt['text']
            dict_txt_lst.append(txtn)
        idwise_json_txt_str=' '.join(dict_txt_lst)
        idwise_json_txt.append(idwise_json_txt_str)
    return idwise_json_txt 
json_txt_extraction()

#updating train data with json text data
df_train['text']=idwise_json_txt

In [ ]:
df_train.head()

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [ ]:
data_title = list(map(clean_text, df_train['dataset_title'].unique()))
data_label = list(map(clean_text, df_train['dataset_label'].unique()))
cleaned_label = list(df_train['cleaned_label'].unique())

In [ ]:
# data_title
# data_label

In [ ]:
labels = set(data_title + data_label + cleaned_label)

In [ ]:
# labels

In [ ]:
# importing test data
#creating id values as list for json data import
id_values_text=sample.Id.values.tolist()

#Creating a function to import json data
idwise_json_txt_test=[]
def json_txt_extraction_test():    
    for id_lst in range(len(id_values_text)):
#         print(id_lst,id_values[id_lst])
        f = open("../input/coleridgeinitiative-show-us-the-data/test/"+ id_values_text[id_lst] +".json",)
        data = json.load(f)
        dict_txt_lst=[]
        for inp_txt in range(len(data)):
            txt=data[inp_txt]
            txtn=txt['text']
            dict_txt_lst.append(txtn)
        idwise_json_txt_str=' '.join(dict_txt_lst)
        idwise_json_txt_test.append(idwise_json_txt_str)
    return idwise_json_txt_test 
json_txt_extraction_test()

#updating train data with json text data
sample['text']=idwise_json_txt_test


In [ ]:
sample

In [ ]:
mapping = dict()
for row in df_train.iterrows():
    cleaned_label = row[1]['cleaned_label'].strip()
    title = clean_text(row[1]['dataset_title']).strip()
    dataSetLabel = clean_text(row[1]['dataset_label']).strip()
    mapTitle = mapping.get(title, set())
    mapTitle.add(cleaned_label)
    mapping[title] = mapTitle
    mapLabel = mapping.get(dataSetLabel, set())
    mapLabel.add(cleaned_label)
    mapping[dataSetLabel] = mapLabel
    mapCleanedLabel = mapping.get(cleaned_label, set())
    mapCleanedLabel.add(cleaned_label)
    mapping[dataSetLabel] = mapLabel

In [ ]:
strings = []
for text in sample['text'].values:
    text = clean_text(text)
    predictString = set()
    for label in labels:
        if label in text:
            for string in mapping[label.strip()]:
                predictString.add(string)
    predictString = "|".join(predictString)
    strings.append(predictString)

In [ ]:
strings

In [ ]:
sample['PredictionString'] = strings
sample.drop(['text'], axis=1).to_csv("submission.csv", index=False)